In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm

# Đọc dữ liệu bất động sản từ file CSV
data = pd.read_csv('../../Data/cleanedData/cleaned_data.csv')

# Loại bỏ các hàng có giá trị thiếu trong cột 'Quận/Huyện' và 'Mức giá'
heatmap_data = data.dropna(subset=['Quận/Huyện', 'Mức giá'])

# Chuyển đổi cột 'Mức giá' thành kiểu số
heatmap_data['Mức giá'] = pd.to_numeric(heatmap_data['Mức giá'], errors='coerce')

# Tính giá trung bình theo quận/huyện
avg_price_by_district = heatmap_data.groupby('Quận/Huyện')['Mức giá'].mean().reset_index()

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/HaNoiMap')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Ghép dữ liệu giá trung bình vào bản đồ Hà Nội
hanoi_map = hanoi_map.merge(avg_price_by_district, left_on='NAME_2', right_on='Quận/Huyện', how='left')

# Tọa độ trung tâm của Hà Nội
hanoi_center = [21.0285, 105.8542]

# Tạo một bản đồ nền bằng folium với nền mờ
m = folium.Map(
    location=hanoi_center,
    zoom_start=10,
    tiles="CartoDB Positron",  # Nền mờ
    max_bounds=True,          # Giữ bản đồ trong giới hạn
    dragging=False,           # Không cho phép kéo bản đồ
    zoom_control=False,       # Tắt nút thu phóng
    scrollWheelZoom=False     # Tắt thu phóng bằng cuộn chuột
)

# Tạo colormap
colormap = cm.linear.YlOrRd_09.scale(hanoi_map['Mức giá'].min(), hanoi_map['Mức giá'].max())

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Mức giá']) if feature['properties']['Mức giá'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Mức giá'],
        aliases=['Quận/Huyện', 'Giá trung bình (triệu VND)'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)

# Thêm colormap vào bản đồ
colormap.add_to(m)

# Lưu và hiển thị bản đồ
m


f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'HaNoiMap': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(


In [3]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm

# Đọc dữ liệu bất động sản từ file CSV
data = pd.read_csv('../../Data/cleanedData/cleaned_data.csv')

# Loại bỏ các hàng có giá trị thiếu trong cột 'Xã/Phường' và 'Mức giá'
heatmap_data = data.dropna(subset=['Xã/Phường', 'Mức giá'])

# Chuyển đổi cột 'Mức giá' thành kiểu số
heatmap_data['Mức giá'] = pd.to_numeric(heatmap_data['Mức giá'], errors='coerce')

# Tính giá trung bình theo xã/phường
avg_price_by_ward = heatmap_data.groupby('Xã/Phường')['Mức giá'].mean().reset_index()

# Đọc shapefile của Việt Nam
vietnam_map = gpd.read_file('../Map/commune map')

def calculate_zoom(area_km2):
    """
    Xác định mức độ thu phóng (zoom level) dựa trên diện tích (km²).
    """
    if area_km2 > 500:
        return 10 # Quận/huyện lớn
    elif area_km2 > 100:
        return 12  # Quận/huyện trung bình
    elif area_km2 > 10:
        return 13  # Quận/huyện nhỏ
    else:
        return 14  # Quận/huyện rất nhỏ

def create_district_map(district_name):
    """
    Tạo bản đồ nhiệt cho quận/huyện được chọn, với zoom phù hợp dựa trên diện tích.
    """
    # Lọc bản đồ cho quận/huyện được chọn
    district_map = vietnam_map[(vietnam_map['NAME_1'] == 'Hà Nội') & (vietnam_map['NAME_2'] == district_name)]
    
    if district_map.empty:
        print(f"Không tìm thấy dữ liệu cho quận/huyện: {district_name}")
        return None
    
    # Tính diện tích (km²) của quận/huyện
    district_map = district_map.to_crs(epsg=3395)  # Chuyển hệ tọa độ để tính diện tích chính xác
    total_area_km2 = district_map['geometry'].area.sum() / 1e6  # Diện tích tổng cộng (m² -> km²)

    # Xác định mức độ thu phóng
    zoom_level = calculate_zoom(total_area_km2)

    # Ghép dữ liệu giá trung bình vào bản đồ
    district_map = district_map.to_crs(epsg=4326)  # Chuyển lại về hệ tọa độ địa lý
    district_map = district_map.merge(avg_price_by_ward, left_on='NAME_3', right_on='Xã/Phường', how='left')

    # Tọa độ trung tâm của quận/huyện
    district_center = [district_map.geometry.centroid.y.mean(), district_map.geometry.centroid.x.mean()]

    # Tạo bản đồ nền
    m_district = folium.Map(
        location=district_center,
        zoom_start=zoom_level,
        tiles="CartoDB Positron",  # Nền mờ
        max_bounds=True,
        dragging=True,  # Cho phép kéo bản đồ
        zoom_control=False,  # Bật nút thu phóng
        scrollWheelZoom=True  # Bật cuộn chuột để phóng to
    )

    # Tạo colormap
    colormap = cm.linear.YlOrRd_09.scale(
        district_map['Mức giá'].min(), 
        district_map['Mức giá'].max()
    )
    # Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
    folium.GeoJson(
        district_map,
        style_function=lambda feature: {
            'fillColor': colormap(feature['properties']['Mức giá']) if feature['properties']['Mức giá'] else 'lightblue',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        },
        tooltip=GeoJsonTooltip(
            fields=['NAME_3', 'Mức giá'],
            aliases=['Xã/Phường', 'Giá trung bình (triệu VND)'],
            localize=True
        ),
        highlight_function=lambda feature: {
            'fillColor': 'yellow',
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.7
        }
    ).add_to(m_district)

    # Thêm colormap vào bản đồ
    colormap.add_to(m_district)

    return m_district

# Ví dụ: tạo bản đồ cho quận Chương Mỹ
district_name = "Hà Đông"  # Thay đổi tên quận/huyện tại đây
map_output = create_district_map(district_name)

# Hiển thị bản đồ
map_output


C:\Users\Admin\AppData\Local\Temp\ipykernel_1284\3360377174.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  district_center = [district_map.geometry.centroid.y.mean(), district_map.geometry.centroid.x.mean()]


In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm
import json
import unicodedata

# Hàm tính sự tăng giảm giá theo thời gian (phần trăm thay đổi)
def calculate_price_change(data, months):
    price_columns = [col for col in data.keys() if col.startswith('Giá')]
    selected_prices = price_columns[-months:]  # Lấy dữ liệu của các tháng gần nhất
    
    # Lấy giá của tháng đầu tiên và tháng cuối cùng
    initial_price = pd.DataFrame([data]).loc[:, selected_prices].iloc[0, 0]
    final_price = pd.DataFrame([data]).loc[:, selected_prices].iloc[0, -1]
    
    # Tính phần trăm thay đổi
    price_change = ((final_price - initial_price) / initial_price) * 100
    return price_change

# Hàm chuẩn hóa tên quận/huyện (loại bỏ dấu, chuyển về chữ thường)
def normalize_district_name(name):
    # Loại bỏ dấu (các ký tự có dấu sẽ được chuyển thành không dấu)
    name_without_accent = ''.join(
        (c for c in unicodedata.normalize('NFD', name) if unicodedata.category(c) != 'Mn')
    )
    # Chuyển 'đ' thành 'd'
    name_without_accent = name_without_accent.replace('đ', 'd').replace('Đ', 'd')
    # Chuyển thành chữ thường và thay thế khoảng trắng thành dấu gạch dưới
    normalized_name = name_without_accent.lower().replace(" ", "_")
    return normalized_name

# Đường dẫn thư mục chứa các file JSON
json_folder_path = r'../../Data/Json/Segment/area/'

# Tính toán sự thay đổi giá theo quận/huyện
district_price_changes = {}
time_range = 12  # Thay đổi thành 6, 12, hoặc 24 để tính cho 6 tháng, 1 năm, hoặc 2 năm

# Đọc dữ liệu từ các file JSON
for file_name in os.listdir(json_folder_path):
    if file_name.endswith('.json'):  # Lọc file JSON
        district_name = os.path.splitext(file_name)[0]  # Lấy tên quận/huyện từ tên file
        normalized_district_name = normalize_district_name(district_name)  # Chuẩn hóa tên quận/huyện
        json_file_path = os.path.join(json_folder_path, file_name)
        
        # Đọc dữ liệu JSON
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)[0]  # Giả định mỗi file chứa một đối tượng JSON
        
        # Tính sự thay đổi giá
        price_change = calculate_price_change(json_data, time_range)
        district_price_changes[normalized_district_name] = price_change

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/HaNoiMap')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Chuẩn hóa tên các quận/huyện trong shapefile Hà Nội
hanoi_map['normalized_NAME_2'] = hanoi_map['NAME_2'].apply(lambda x: normalize_district_name(x))

# Gắn dữ liệu sự thay đổi giá vào GeoDataFrame bằng cách ghép tên đã chuẩn hóa
hanoi_map['Price_Change'] = hanoi_map['normalized_NAME_2'].map(district_price_changes)

# Tọa độ trung tâm của Hà Nội
hanoi_center = [21.0285, 105.8542]

# Tạo một bản đồ nền bằng folium
m = folium.Map(
    location=hanoi_center,
    zoom_start=10,
    tiles="CartoDB Positron", 
    dragging=False,           # Không cho phép kéo bản đồ
    zoom_control=False,       # Tắt nút thu phóng
    scrollWheelZoom=False
)

# Tạo colormap
colormap = cm.linear.RdYlGn_09.scale(
    hanoi_map['Price_Change'].min(),
    hanoi_map['Price_Change'].max()
)

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Price_Change']) if feature['properties']['Price_Change'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Price_Change'],
        aliases=['Quận/Huyện', 'Sự thay đổi giá (%)'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)

# Thêm colormap vào bản đồ
colormap.add_to(m)

# Lưu và hiển thị bản đồ
m


f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'HaNoiMap': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setite

In [28]:
import os
import json
import folium
import geopandas as gpd
from folium.features import GeoJsonTooltip
import branca.colormap as cm
import unicodedata

# Hàm chuẩn hóa tên quận/huyện
def normalize_district_name(name):
    name_without_accent = ''.join(
        (c for c in unicodedata.normalize('NFD', name) if unicodedata.category(c) != 'Mn')
    )
    name_without_accent = name_without_accent.replace('đ', 'd').replace('Đ', 'd')
    normalized_name = name_without_accent.lower().replace(" ", "_")
    return normalized_name

# Đường dẫn thư mục chứa các file JSON
json_folder_path = r'../../Data/Json/Segment/area/'

# Lưu dữ liệu phân khúc giá cho từng quận/huyện
district_price_segments = {}

# Đọc và xử lý các file JSON
for file_name in os.listdir(json_folder_path):
    if file_name.endswith('.json'):
        json_file_path = os.path.join(json_folder_path, file_name)
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)  # Giả định mỗi file chứa một đối tượng JSON
            
            # Lấy tên quận/huyện và phân khúc giá
            area_name = json_data.get('area_name', 'Unknown Area')
            segments = {
                'Dưới 50 triệu': json_data.get('under_50', 0),
                'Từ 50 triệu đến 100 triệu': json_data.get('between_50_100', 0),
                'Từ 100 triệu đến 150 triệu': json_data.get('between_100_150', 0),
                'Từ 150 triệu đến 200 triệu': json_data.get('between_150_200', 0),
                'Trên 200 triệu': json_data.get('over_200', 0)
            }
            
            # Lưu dữ liệu phân khúc cho từng quận/huyện
            normalized_district_name = normalize_district_name(area_name)
            district_price_segments[normalized_district_name] = segments

# Chọn phân khúc bạn muốn hiển thị (ví dụ: "Dưới 50 triệu")
selected_index = "Từ 50 triệu đến 100 triệu"

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/HaNoiMap')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Chuẩn hóa tên các quận/huyện trong shapefile Hà Nội
hanoi_map['normalized_NAME_2'] = hanoi_map['NAME_2'].apply(lambda x: normalize_district_name(x))

# Gắn dữ liệu phân khúc vào GeoDataFrame
hanoi_map['Price_Segment'] = hanoi_map['normalized_NAME_2'].map(
    lambda district: district_price_segments.get(district, {}).get(selected_index, 0)
)

# Tọa độ trung tâm của Hà Nội
hanoi_center = [21.0285, 105.8542]

# Tạo bản đồ với Folium
m = folium.Map(location=hanoi_center, zoom_start=10, tiles="CartoDB Positron", dragging=False, zoom_control=False, scrollWheelZoom=False)

# Tạo colormap cho phân khúc
colormap = cm.linear.RdYlGn_09.scale(hanoi_map['Price_Segment'].min(), hanoi_map['Price_Segment'].max())

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Price_Segment']) if feature['properties']['Price_Segment'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Price_Segment'],
        aliases=['Quận/Huyện', f'{selected_index} (%)'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)

# Thêm colormap vào bản đồ
colormap.add_to(m)

# Lưu và hiển thị bản đồ
m


f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'HaNoiMap': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setite

In [14]:
import os
import json
import folium
import geopandas as gpd
from folium.features import GeoJsonTooltip
import branca.colormap as cm
import unicodedata

# Hàm chuẩn hóa tên quận/huyện
def normalize_district_name(name):
    name_without_accent = ''.join(
        (c for c in unicodedata.normalize('NFD', name) if unicodedata.category(c) != 'Mn')
    )
    name_without_accent = name_without_accent.replace('đ', 'd').replace('Đ', 'd')
    normalized_name = name_without_accent.lower().replace(" ", "_")
    return normalized_name

# Biến để chọn loại bất động sản
selected_property_type = "Nhà riêng"  # Ví dụ: "Căn hộ chung cư", "Nhà mặt phố", "Chung cư mini, căn hộ dịch vụ"

# Đường dẫn thư mục chứa các file JSON
json_folder_path = r'../../Data/Json/Number_Of_Type_Property/area/'

# Lưu dữ liệu cho từng quận/huyện
district_property_count = {}

# Đọc và xử lý các file JSON
for file_name in os.listdir(json_folder_path):
    if file_name.endswith('.json'):
        json_file_path = os.path.join(json_folder_path, file_name)
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)  # Giả định mỗi file chứa một đối tượng JSON
            
            # Lấy tên quận/huyện từ tên file (bỏ đuôi ".json")
            district_name = os.path.splitext(file_name)[0]
            
            # Lấy số lượng của loại bất động sản được chọn
            property_count = json_data.get(selected_property_type, 0)
            
            # Lưu dữ liệu vào từ điển
            district_property_count[normalize_district_name(district_name)] = property_count

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/HaNoiMap')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Chuẩn hóa tên các quận/huyện trong shapefile Hà Nội
hanoi_map['normalized_NAME_2'] = hanoi_map['NAME_2'].apply(lambda x: normalize_district_name(x))

# Gắn dữ liệu loại bất động sản vào GeoDataFrame
hanoi_map['Property_Count'] = hanoi_map['normalized_NAME_2'].map(
    lambda district: district_property_count.get(district, 0)
)

# Tọa độ trung tâm của Hà Nội
hanoi_center = [21.0285, 105.8542]

# Tạo bản đồ với Folium
m = folium.Map(location=hanoi_center, zoom_start=10, tiles="CartoDB Positron", dragging=False, zoom_control=False, scrollWheelZoom=False)

# Tạo colormap cho số lượng bất động sản
colormap = cm.linear.YlOrRd_09.scale(hanoi_map['Property_Count'].min(), hanoi_map['Property_Count'].max())

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Property_Count']) if feature['properties']['Property_Count'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Property_Count'],
        aliases=['Quận/Huyện', f'Số lượng {selected_property_type}'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)

# Thêm colormap vào bản đồ
colormap.add_to(m)

# Lưu và hiển thị bản đồ
m


f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'HaNoiMap': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setite

In [16]:
import os
import json
import folium
import geopandas as gpd
from folium.features import GeoJsonTooltip
import branca.colormap as cm
import unicodedata

# Hàm chuẩn hóa tên quận/huyện
def normalize_district_name(name):
    name_without_accent = ''.join(
        (c for c in unicodedata.normalize('NFD', name) if unicodedata.category(c) != 'Mn')
    )
    name_without_accent = name_without_accent.replace('đ', 'd').replace('Đ', 'd')
    normalized_name = name_without_accent.lower().replace(" ", "_")
    return normalized_name

# Biến để chọn loại bất động sản
selected_property_type = "Nhà riêng"  # Ví dụ: "Căn hộ chung cư", "Nhà mặt phố", "Chung cư mini, căn hộ dịch vụ"

# Đường dẫn thư mục chứa các file JSON
json_folder_path = r'../../Data/Json/Mean_Price/area/'

# Lưu dữ liệu giá trung bình cho từng quận/huyện
district_mean_price = {}

# Đọc và xử lý các file JSON
for file_name in os.listdir(json_folder_path):
    if file_name.endswith('.json'):
        json_file_path = os.path.join(json_folder_path, file_name)
        with open(json_file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)  # Giả định mỗi file chứa một đối tượng JSON
            
            # Lấy tên quận/huyện từ tên file (bỏ đuôi ".json")
            district_name = os.path.splitext(file_name)[0]
            
            # Lấy giá trung bình của loại bất động sản được chọn
            mean_price = json_data.get(selected_property_type, 0)
            
            # Lưu dữ liệu vào từ điển
            district_mean_price[normalize_district_name(district_name)] = mean_price

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/HaNoiMap')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Chuẩn hóa tên các quận/huyện trong shapefile Hà Nội
hanoi_map['normalized_NAME_2'] = hanoi_map['NAME_2'].apply(lambda x: normalize_district_name(x))

# Gắn dữ liệu giá trung bình vào GeoDataFrame
hanoi_map['Mean_Price'] = hanoi_map['normalized_NAME_2'].map(
    lambda district: district_mean_price.get(district, 0)
)

# Tọa độ trung tâm của Hà Nội
hanoi_center = [21.0285, 105.8542]

# Tạo bản đồ với Folium
m = folium.Map(location=hanoi_center, zoom_start=10, tiles="CartoDB Positron", dragging=False, zoom_control=False, scrollWheelZoom=False)

# Tạo colormap cho giá trung bình
colormap = cm.linear.YlGnBu_09.scale(hanoi_map['Mean_Price'].min(), hanoi_map['Mean_Price'].max())

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Mean_Price']) if feature['properties']['Mean_Price'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Mean_Price'],
        aliases=['Quận/Huyện', f'Giá trung bình {selected_property_type} (triệu/m²)'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)

# Thêm colormap vào bản đồ
colormap.add_to(m)

# Lưu và hiển thị bản đồ
m.save("heatmap\Average_price_of_type.html")


<>:90: SyntaxWarning: invalid escape sequence '\A'
<>:90: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Admin\AppData\Local\Temp\ipykernel_10328\3085763203.py:90: SyntaxWarning: invalid escape sequence '\A'
  m.save("heatmap\Average_price_of_type.html")
f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'HaNoiMap': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
f:\anaconda\envs\BTL\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to

FileNotFoundError: [Errno 2] No such file or directory: 'heatmap\\Average_price_of_type.html'